In [1]:
import sys, os

from torch.utils.tensorboard import SummaryWriter
from absl import flags
from absl import app

from feature_extractor.utils import save_to_logs, get_train_dir
from feature_extractor.emb_model_lib import EmbeddingModel

import Dataset as ds
import torch

2023-06-27 14:59:05.955987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [3]:
param = {
    "L2D": { # Parameter for Learning to defer
        "maxLabels": 16,
    },
    "TARGET": "Airspace_Opacity",
    "PATH": "../../Datasets/NIH/",
    "K": 10, #Number of folds
    "LABELER_IDS": [4323195249, 4295232296],
    
    "batch_size": 64,
    "alpha": 1.0, #scaling parameter for the loss function, default=1.0
    "epochs": 150,
    "patience": 50, #number of patience steps for early stopping the training
    "expert_type": "MLPMixer", #specify the expert type. For the type of experts available, see-> models -> experts. defualt=predict
    "n_classes": 2, #K for K class classification
    "k": 0, #
    "n_experts": 2, #
    "lr": 0.001, #learning rate
    "weight_decay": 5e-4, #
    "warmup_epochs": 5, #
    "loss_type": "softmax", #surrogate loss type for learning to defer
    "ckp_dir": "./Models", #directory name to save the checkpoints
    "experiment_name": "multiple_experts", #specify the experiment name. Checkpoints will be saved with this name
    #
    "TRAIN_BATCH_SIZE": 64,
    "TEST_BATCH_SIZE": 64,
    "NUM_EXPERTS": 2,

    "PRELOAD": True,
    "PREPROCESS": False,
    "PREBUILD": True,

    "GT": True, # Determines if the classifier gets all data with GT Label or only the labeld data
}

In [4]:
import neptune

import json

#with open('neptune_config.json', 'r') as f:
#    config = json.load(f)

#config_neptune = config["neptune"]
"""run = neptune.init_run(
    project=config_neptune["project"],
    api_token=config_neptune["api_token"],
    custom_run_id="AI"
)"""

NEPTUNE = False

param = {
    "TARGET": "Airspace_Opacity",
    "PATH": "../../Datasets/NIH/",
    "K": 10, #Number of folds
    "LABELER_IDS": [4323195249, 4295232296],
    
    "batch_size": 64,
    "alpha": 1.0, #scaling parameter for the loss function, default=1.0
    #"epochs": 50,
    "epochs": 50,
    "patience": 15, #number of patience steps for early stopping the training
    "expert_type": "MLPMixer", #specify the expert type. For the type of experts available, see-> models -> experts. defualt=predict
    "n_classes": 2, #K for K class classification
    "K": 10, #
    
    "TRAIN_BATCH_SIZE": 64,
    "TEST_BATCH_SIZE": 64,
    "NUM_EXPERTS": 2,

    "PRELOAD": True,
    "PREBUILD": True,
    
    "OVERLAP K": 8,
    "NUMBER LABELS": 8,
}

dataManager = ds.DataManager(path = param["PATH"], target = param["TARGET"], param=param, seeds=[0])
dataManager.createData()
sslDataset = dataManager.getSSLDataset(0)
train_dataloader, val_dataloader, test_dataloader = sslDataset.get_data_loader_for_fold(0)
#train_dataloader, val_dataloader, test_dataloader = nih_dataloader.get_data_loader_for_fold(1)
dataloaders = (train_dataloader, val_dataloader, test_dataloader)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Loaded image number: 0
Loaded image number: 200
Loaded image number: 400
Loaded image number: 600
Loaded image number: 800
Loaded image number: 1000
Loaded image number: 1200
Loaded image number: 1400
Loaded image number: 1600
Loaded image number: 1800
Loaded image number: 2000
Loaded image number: 2200
Loaded image number: 2400
Loaded image number: 2600
Loaded image number: 2800
Loaded image number: 3000
Loaded image number: 3200
Loaded image number: 3400
Loaded image number: 3600
Loaded image number: 3800
Loaded image number: 4000
Loaded image number: 4200
Full length: 4381
Loaded image number: 0
Loaded image number: 200
Loaded image number: 400
Loaded image number: 600
Loaded image number: 800


/home/joli/joli-env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Length of train + test + val: 852
Loaded set number 0
Loaded set number 1
Loaded set number 2
Loaded set number 3
Loaded set number 4
Loaded set number 5
Loaded set number 6
Loaded set number 7
Loaded set number 8
Loaded set number 9
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added


In [5]:
device

device(type='cuda', index=0)

In [6]:
args = {
    'dataset': "nih",
    'model': "resnet18",
    'num_classes': 2,
    'batch': 64,
    'lr': 0.001,
}

path = "../../../Datasets/NIH/"

wkdir = os.getcwd()
sys.path.append(wkdir)
    
# get training directory
train_dir = get_train_dir(wkdir, args, 'emb_net')

print("Train dir: " + train_dir)
# initialize summary writer for tensorboard
writer = SummaryWriter(train_dir + 'logs/')

# initialize base model
emb_model = EmbeddingModel(args, wkdir, writer, dataloaders, param)
# try to load previous training runs
start_epoch = emb_model.load_from_checkpoint(mode='latest')
# train model
for epoch in range(start_epoch, 100):
    # train one epoch
    loss = emb_model.train_one_epoch(epoch)
    # get validation accuracy
    valid_acc = emb_model.get_test_accuracy(return_acc=True)
    print(f'loss: {loss}')
    # save logs to json
    save_to_logs(train_dir, valid_acc, loss.item())
    # save model to checkpoint
    emb_model.save_to_checkpoint(epoch, loss, valid_acc)
# get test accuracy
acc = emb_model.get_test_accuracy()

if NEPTUNE:
    run["embedded/test/accuracy"].append(acc)

Train dir: /home/joli/Masterarbeit/SSL/NIH/emb_net@dataset-nih-model-resnet18-num_classes-2/
load Resnet-18 checkpoint
load Resnet-18 pretrained on ImageNet
Loaded Model resnet18


/home/joli/joli-env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/joli/joli-env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Found latest checkpoint at /home/joli/Masterarbeit/SSL/NIH/emb_net@dataset-nih-model-resnet18-num_classes-2//checkpoints/checkpoint.latest
Continuing in epoch 5
Train Epoch 5: |███████████████████████████████████████-| 100.0% Complete
Test-Accuracy: 0.7911464245175936 
Test-Acc-Class [0.80941704 0.77241379]
loss: 0.44786012172698975
Train Epoch 6: |███████████████████████████████████████-| 100.0% Complete
Test-Accuracy: 0.7956867196367764 
Test-Acc-Class [0.78923767 0.80229885]
loss: 0.4517754912376404
Train Epoch 7: |███████████████████████████████████████-| 100.0% Complete
Test-Accuracy: 0.7922814982973894 
Test-Acc-Class [0.80044843 0.78390805]
loss: 0.5286190509796143
Train Epoch 8: |███████████████████████████████████████-| 100.0% Complete
Test-Accuracy: 0.7900113507377979 
Test-Acc-Class [0.80269058 0.77701149]
loss: 0.44550326466560364
Train Epoch 9: |███████████████████████████████████████-| 100.0% Complete
Test-Accuracy: 0.793416572077185 
Test-Acc-Class [0.79820628 0.78850575